In [ ]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/Yolo_results/dataset.zip"
extract_path = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(" Giải nén xong!")


 Giải nén xong!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install -q albumentations opencv-python-headless


In [ ]:
import albumentations as A
import cv2
import os
import shutil
from tqdm import tqdm

# Augmentation thời tiết và ánh sáng
weather_aug = A.Compose([
    A.RandomRain(p=0.3),
    A.RandomFog(p=0.3),
    A.RandomSnow(p=0.2),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.1), contrast_limit=0.2, p=0.5),
    A.RandomShadow(p=0.3),
])

# Thư mục gốc
img_input = "/content/dataset/dataset/images/train"
img_aug = "/content/dataset/dataset/images/train_aug"
lbl_input = "/content/dataset/dataset/labels_Labelimg/train"
lbl_aug = "/content/dataset/dataset/labels/train_aug"

os.makedirs(img_aug, exist_ok=True)
os.makedirs(lbl_aug, exist_ok=True)


In [ ]:
for fname in tqdm(os.listdir(img_input)):
    if fname.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(img_input, fname)
        label_path = os.path.join(lbl_input, fname.replace('.jpg', '.txt').replace('.png', '.txt'))

        # Đọc ảnh
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Tạo ảnh augment
        aug = weather_aug(image=image)
        aug_img = aug['image']
        aug_img = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)

        # Lưu ảnh
        aug_img_path = os.path.join(img_aug, "aug_" + fname)
        cv2.imwrite(aug_img_path, aug_img)

        # Copy nhãn tương ứng
        if os.path.exists(label_path):
            aug_label_path = os.path.join(lbl_aug, "aug_" + fname.replace('.jpg', '.txt').replace('.png', '.txt'))
            shutil.copy(label_path, aug_label_path)


100%|██████████| 6574/6574 [12:53<00:00,  8.50it/s]


In [ ]:
# Tạo thư mục train_all
img_train_all = "/content/dataset/dataset/images/train_all"
lbl_train_all = "/content/dataset/dataset/labels_Labelimg/train_all"
os.makedirs(img_train_all, exist_ok=True)
os.makedirs(lbl_train_all, exist_ok=True)

# Copy ảnh gốc
for fname in os.listdir(img_input):
    shutil.copy(os.path.join(img_input, fname), os.path.join(img_train_all, fname))

# Copy ảnh augment
for fname in os.listdir(img_aug):
    shutil.copy(os.path.join(img_aug, fname), os.path.join(img_train_all, fname))

# Copy nhãn gốc
for fname in os.listdir(lbl_input):
    shutil.copy(os.path.join(lbl_input, fname), os.path.join(lbl_train_all, fname))

# Copy nhãn augment
for fname in os.listdir(lbl_aug):
    shutil.copy(os.path.join(lbl_aug, fname), os.path.join(lbl_train_all, fname))


In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv


class ChannelAttention(nn.Module):
    def __init__(self, in_channels, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.shared_mlp = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_channels // ratio, in_channels, 1, bias=False),
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.shared_mlp(self.avg_pool(x))
        max_out = self.shared_mlp(self.max_pool(x))
        return x * self.sigmoid(avg_out + max_out)


class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1)
        return x * self.sigmoid(self.conv(concat))


class CBAM(nn.Module):
    def __init__(self, in_channels, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, ratio)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x


class CBAMBlock(nn.Module):
    def __init__(self, c1, c2):
        super().__init__()
        self.conv = Conv(c1, c2)
        self.cbam = CBAM(c2)

    def forward(self, x):
        return self.cbam(self.conv(x))


class CustomYOLO(YOLO):
    def __init__(self, cfg='yolov8m.yaml'):
        super().__init__(cfg)
        self.modify_model()

    def modify_model(self):
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, Conv):
                c1, c2 = layer.conv.in_channels, layer.conv.out_channels
                self.model.model[i] = CBAMBlock(c1, c2)


model = CustomYOLO('yolov8m.pt')


print(model)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 49.7M/49.7M [00:00<00:00, 311MB/s]


CustomYOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): CBAMBlock(
        (conv): Conv(
          (conv): Conv2d(3, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (cbam): CBAM(
          (channel_attention): ChannelAttention(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (max_pool): AdaptiveMaxPool2d(output_size=1)
            (shared_mlp): Sequential(
              (0): Conv2d(48, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): ReLU()
              (2): Conv2d(3, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
            (sigmoid): Sigmoid()
          )
          (spatial_attention): SpatialAttention(
            (conv): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
            (sigmoid): Sigmoid()
          )
        )
      )
      (

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO("yolov8n.pt")

# Train model
model.train(
    data="/content/dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    workers=4,
    project="yolov8_train",
    name="exp1"
)


Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=yolov8_train, name=exp12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

100%|██████████| 755k/755k [00:00<00:00, 20.9MB/s]


Overriding model.yaml nc=80 with nc=8

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 99.3MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      36/50      3.22G      1.134      0.746      1.026         48        640: 100%|██████████| 411/411 [01:54<00:00,  3.60it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:13<00:00,  3.73it/s]

                   all       1644       5310       0.77      0.755      0.822      0.578



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      37/50      3.24G      1.119     0.7387       1.02         70        640:  78%|███████▊  | 319/411 [01:26<00:37,  2.48it/s]

In [ ]:
ls /content/runs/detect/


ls: cannot access '/content/runs/detect/': No such file or directory


In [ ]:
from ultralytics import YOLO
from google.colab import drive
import os


# Định nghĩa thư mục lưu trên Drive
drive_save_dir = "/content/drive/MyDrive/Yolo_results"

# Tạo thư mục nếu chưa có
os.makedirs(drive_save_dir, exist_ok=True)

# Cập nhật đường dẫn train mới trong data.yaml
yaml_path = "/content/dataset/dataset/data.yaml"

# Load model YOLO
model = YOLO("yolov8n.pt")

# Train model với tập dữ liệu đã được augment + lưu kết quả vào Google Drive
model.train(
    data=yaml_path,
    epochs=30,
    imgsz=640,
    batch=16,
    device=0,
    workers=4,
    project=drive_save_dir,
    name="exp3"
)


Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/dataset/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=/content/drive/MyDrive/Yolo_results, name=exp34, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_la

train: Scanning /content/dataset/dataset/labels/train_all... 13148 images, 28 backgrounds, 0 corrupt: 100%|██████████| 13148/13148 [00:11<00:00, 1156.74it/s]

train: WARNING ⚠️ /content/dataset/dataset/images/train_all/aug_highway_2784_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/dataset/images/train_all/aug_highway_3297_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/dataset/images/train_all/highway_2784_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/dataset/images/train_all/highway_3297_2020-08-26.jpg: 1 duplicate labels removed


train: New cache created: /content/dataset/dataset/labels/train_all.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/dataset/dataset/labels/val... 1644 images, 4 backgrounds, 0 corrupt: 100%|██████████| 1644/1644 [00:01<00:00, 826.71it/s] 


...

                   all       1644       5310      0.791      0.778       0.84      0.599



30 epochs completed in 2.152 hours.
Optimizer stripped from /content/drive/MyDrive/Yolo_results/exp34/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Yolo_results/exp34/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Yolo_results/exp34/weights/best.pt...
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:18<00:00,  2.76it/s]


                   all       1644       5310      0.789      0.789      0.841      0.601
                  auto        144        159      0.855      0.862      0.926      0.588
                   bus        261        282      0.836      0.816      0.863      0.669
                   car       1164       2315      0.903      0.904      0.954      0.677
                   lcv        199        209        0.6       0.56      0.654      0.488
            motorcycle        820       1503      0.829      0.779      0.865      0.484
             multiaxle        217        234      0.752      0.777      0.812      0.657
               tractor         31         31      0.837      0.806      0.854      0.643
                 truck        525        577      0.703      0.806      0.801      0.602
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /content/drive/MyDrive/Yolo_results/exp34


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c9eda93e290>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import os
# Định nghĩa thư mục lưu trên Drive
drive_save_dir = "/content/drive/MyDrive/Yolo_results"

# Tạo thư mục nếu chưa có
os.makedirs(drive_save_dir, exist_ok=True)

# Cập nhật đường dẫn train mới trong data.yaml
yaml_path = "/content/dataset/content/dataset/dataset/data.yaml"

# Khởi tạo mô hình CustomYOLO đã thêm CBAM
model = CustomYOLO("yolov8m.pt")


model.train(
    data=yaml_path,
    epochs=20,
    imgsz=640,
    batch=16,
    device=0,
    workers=4,
    project=drive_save_dir,
    name="exp4"
)

Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/dataset/content/dataset/dataset/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=/content/drive/MyDrive/Yolo_results, name=exp43, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

train: Scanning /content/dataset/content/dataset/dataset/labels/train_all.cache... 13148 images, 28 backgrounds, 0 corrupt: 100%|██████████| 13148/13148 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/content/dataset/dataset/images/train_all/aug_highway_2784_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/content/dataset/dataset/images/train_all/aug_highway_3297_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/content/dataset/dataset/images/train_all/highway_2784_2020-08-26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/content/dataset/dataset/images/train_all/highway_3297_2020-08-26.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/content/dataset/dataset/labels/val.cache... 1644 images, 4 backgrounds, 0 corrupt: 100%|██████████| 1644/1644 [00:00<?, ?it/s]


Plotting labels to /content/drive/MyDrive/Yolo_results/exp43/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Yolo_results/exp43
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1644       5310      0.776      0.788      0.843      0.599



20 epochs completed in 2.592 hours.
Optimizer stripped from /content/drive/MyDrive/Yolo_results/exp43/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/Yolo_results/exp43/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/Yolo_results/exp43/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,844,392 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 52/52 [00:28<00:00,  1.80it/s]


                   all       1644       5310      0.775      0.788      0.843      0.599
                  auto        144        159       0.81       0.83      0.895      0.572
                   bus        261        282      0.811      0.826      0.873      0.671
                   car       1164       2315      0.883      0.915      0.956      0.675
                   lcv        199        209      0.607      0.569      0.653      0.489
            motorcycle        820       1503      0.817      0.788      0.872      0.489
             multiaxle        217        234      0.722      0.739      0.824      0.673
               tractor         31         31      0.839      0.838      0.866      0.623
                 truck        525        577      0.709      0.799      0.801      0.603
Speed: 0.2ms preprocess, 9.9ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/drive/MyDrive/Yolo_results/exp43


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a561f5c9e50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,

In [ ]:
!zip -r dataset.zip /content/dataset/dataset


Streaming output truncated to the last 5000 lines.
  adding: content/dataset/dataset/labels/val/train_all/Highway_1120_2020-07-30_jpg.rf.a218515e083d0f71222b1ba6e5a95403.txt (deflated 44%)
  adding: content/dataset/dataset/labels/val/train_all/aug_Highway_813_2020-07-30_jpg.rf.6741fda21129a79ee823dbd741ca919f.txt (deflated 38%)
  adding: content/dataset/dataset/labels/val/train_all/highway_2973_2020-08-26.txt (deflated 26%)
  adding: content/dataset/dataset/labels/val/train_all/aug_Highway_83_2020-07-30_jpg.rf.4307767db9c6aad2ce53d3e6718bb947.txt (deflated 38%)
  adding: content/dataset/dataset/labels/val/train_all/Highway_1026_2020-07-30_jpg.rf.5f71f229ed2254057e61ec71523a7500.txt (deflated 46%)
  adding: content/dataset/dataset/labels/val/train_all/aug_Highway_1133_2020-07-30_jpg.rf.0f8687a658d46b4b7eb365ba5ffbe569.txt (deflated 38%)
  adding: content/dataset/dataset/labels/val/train_all/Highway_822_2020-07-30_jpg.rf.19b6a8b254db88e09193a0d5fe9c98d0.txt (deflated 27%)
  adding: conte